# Assignment 3: Predictive analysis - Part 1: Linear Regression
## Group 105
- Natasa Bolic (300241734)
- Brent Palmer (300193610)
## Imports

In [4]:
# imports
import pandas as pd

## Introduction

## Dataset Description

## Regression Empirical Study

## Conclusion

## References